In [1]:
import os
import pandas as pd
from tqdm import tqdm

## Get all datasets properties and store them

### Code

The following code **iterates over all files** in the root directory containing all SHARE datasets. It extracts the properties of each dataset and stores them in a pandas dataframe. The properties are:
- dataset name
- wave
- number of rows
- number of columns
- column names

### Details

- For some reason, one or multiple files, when trying to read them, throw a `ValueError`. However, `pandas` gives us the solution by telling us we should add `convert_categoricals=False` to the `read_csv` function when this is happening. This is the reason why we have a `try` and `except` block in the code.

In [4]:
# initiate constants
directory = 'data/'
file_names = []
waves = []
n_rows = []
n_columns = []
columns = []

# iterate through all files in the directory
for root, dirs, files in tqdm(os.walk(directory)):
    for file in files:
        if file.endswith('.dta'):
            try:
                dataset = pd.read_stata(os.path.join(root, file))
            except ValueError:
                dataset = pd.read_stata(os.path.join(root, file), convert_categoricals=False)
            file_names.append(file)
            waves.append(file[6])
            n_rows.append(len(dataset))
            n_columns.append(len(dataset.columns))
            columns.append(list(dataset.columns))

# create a dataframe with the results
df = pd.DataFrame({
    'file_name': file_names,
    'wave': waves,
    'n_rows': n_rows,
    'n_columns': n_columns,
    'columns': columns
})

10it [01:53, 11.35s/it]


## Discriminate normal and particular datasets

According to the [official documentation](https://share-eric.eu/data/faqs-support):

*"The naming of variables is harmonised across waves. Variable names in the CAPI instrument data use the following format: mmXXXyyy_LL. “mm” is the module identifier, e.g. DN for the demographics module, “XXX” refers to the question number, e.g. 001, and “yyy” are optional digits for dummy variables (indicated by “d”), euro conversion (indicated by “e”) or unfolding brackets (indicated by “ub”). The separation character “_” is followed by “LL” optional digits for category or loop indication (“outer loop”)."*

For this reason, we add **boolean indicator** columns to the dataframe to discriminate normal datasets from particular ones. This allows us to see that approximately 10% of the datasets are particular, and maybe useless for us.

In [7]:
def is_file_normal(file_name: str, only_gv: bool=True) -> bool:
    """
    Detect if the last 3 elements of the string follow the pattern _ab, 
    where 'a' and 'b' are single letters.
    
    Args:
    - file_name (str): The string to be checked.
    Returns:
    - bool: True if the pattern is found, False otherwise.
    """

    # special case the General Variables files
    if '_gv_' in file_name:
        return True
    
    if only_gv:
        return False
    
    # remove file extension
    suffix = file_name[:-4]

    # check if the suffix is long enough
    if len(suffix) < 3:
        return False
    
    # check if the last three elements follow the pattern
    last_three = suffix[-3:]
    if last_three[0] == '_' and last_three[1].isalpha() and last_three[2].isalpha():
        return True
    else:
        return False
    
# filter out the files that are not normal
df['is_normal'] = df['file_name'].apply(is_file_normal, only_gv=False)
df['is_gv'] = df['file_name'].apply(is_file_normal, only_gv=True)

In [8]:
print(round(df.is_normal.value_counts()/len(df)*100,2))
print()
print(round(df.is_gv.value_counts()/len(df)*100,2))

is_normal
True     89.44
False    10.56
Name: count, dtype: float64

is_gv
False    76.24
True     23.76
Name: count, dtype: float64


In [9]:
# save the dataframe to a csv file
df.to_csv('data_info.csv', index=False)
df.sample(10)

,file_name,wave,n_rows,n_columns,columns,is_normal,is_gv
209,sharew8_rel9-0-0_xt.dta,8,4175,202,"[mergeid, hhid8, country, language_xt, gender_...",True,False
128,sharew2_rel9-0-0_technical_variables.dta,2,37132,16,"[mergeid, hhid2, mergeidp2, coupleid2, country...",False,False
278,sharew9_rel9-0-0_xt.dta,9,3491,220,"[mergeid, hhid9, country, language_xt, gender_...",True,False
264,sharew3_rel9-0-0_gv_weights.dta,3,28454,14,"[mergeid, hhid3, mergeidp3, coupleid3, country...",True,True
126,sharew2_rel9-0-0_gv_health.dta,2,37132,44,"[mergeid, hhid2, mergeidp2, coupleid2, country...",True,True
207,sharew8_rel9-0-0_as.dta,8,53695,95,"[mergeid, hhid8, mergeidp8, coupleid8, country...",True,False
266,sharew3_rel9-0-0_dq.dta,3,28454,177,"[mergeid, hhid3, mergeidp3, coupleid3, country...",True,False
1,sharew5_rel9-0-0_gv_imputations.dta,5,330190,279,"[mergeid, hhid5, mergeidp5, coupleid5, country...",True,True
100,sharew2_rel9-0-0_cv_r.dta,2,55262,30,"[mergeid, hhid2, mergeidp2, coupleid2, country...",False,False
101,sharew2_rel9-0-0_xt.dta,2,726,108,"[mergeid, hhid2, country, language_xt, gender_...",True,False


## Get column names for each wave

We store the column names for each wave in `.csv` files, so we can use them later. The aim is that we don't have to open all files just for the columns names.

In [10]:
df = pd.read_csv('data_info.csv')
df = df[df['is_gv'] == True] # only keep the general variables files)

# iterate through the waves
for wave in range(1,9+1):
    columns_properties = pd.DataFrame()
    subset = df[df['wave'] == wave]

    # iterate through the files in the wave
    for columns,filename in zip(subset['columns'],subset['file_name']):
        
        # get all columns and the file they belong to
        row = list(columns[1:-1].replace("'", "").split(', '))
        filenames = [filename]*len(row)

        # add the columns and the file to the dataframe
        temp = pd.DataFrame({'column': row, 'file_name': filenames})
        columns_properties = pd.concat([columns_properties, temp])

    columns_properties.drop_duplicates(inplace=True)
    print(f'Wave {wave} has {len(columns)} unique columns.')

    # remove all column names dupplicate (keep first occurence)
    columns_properties.drop_duplicates(subset='column', keep='first', inplace=True)

    # save as csv file
    columns_properties.to_csv(f'columns/wave_{wave}_columns.csv', index=False)

Wave 1 has 483 unique columns.
Wave 2 has 452 unique columns.
Wave 3 has 151 unique columns.
Wave 4 has 89 unique columns.
Wave 5 has 99 unique columns.
Wave 6 has 2810 unique columns.
Wave 7 has 239 unique columns.
Wave 8 has 199 unique columns.
Wave 9 has 853 unique columns.


In [11]:
test = pd.read_csv('columns/wave_1_columns.csv')
test[test['column']=='language']

,column,file_name
5,language,sharew1_rel9-0-0_gv_weights.dta
